# TCGA-UCEC — Notebook 03
## QC gènes et normalisation des données RNA-seq

**Objectif** : QC gènes + normalisation après harmonisation patient (Notebook 02).

## 1. Imports et configuration

In [1]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import warnings                                 # Masquer warnings (dépréciation)
import gc                                       # Gestion mémoire (nettoyage objets inutilisés)
import json                                     # Lecture du dictionnaire de métadonnées

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

# ==========================================================================================================
PROJECT_ROOT = r"C:\Z\M2_AIDA\TCGA_UCEC_project" #Laïla

DIRS = {
    # Racine data
    "DATA": os.path.join(PROJECT_ROOT, "data"),

    # Données
    "RAW": os.path.join(PROJECT_ROOT, "data", "raw"),

    # Données intermédiaires (persistées)
    "PROCESSED":        os.path.join(PROJECT_ROOT, "data", "processed"),
    "NORM":             os.path.join(PROJECT_ROOT, "data", "processed", "normalized"),
    "QC_FILTERED":      os.path.join(PROJECT_ROOT, "data", "processed", "qc_filtered"),
    "COHORT_FILTERED":  os.path.join(PROJECT_ROOT, "data", "processed", "cohort_filtered"),
    "SC_OBJECTS":       os.path.join(PROJECT_ROOT, "data", "processed", "single_cell_objects"),
    "PSEUDOBULK_PROC":  os.path.join(PROJECT_ROOT, "data", "processed", "pseudobulk"),

    # Artefacts (par étape du pipeline)
    "ARTEFACTS": os.path.join(PROJECT_ROOT, "data", "artefacts"),

    "EDA":         os.path.join(PROJECT_ROOT, "data", "artefacts", "exploratory_data_analysis"),
    "QC":          os.path.join(PROJECT_ROOT, "data", "artefacts", "qc_analysis"),
    "COHORT":      os.path.join(PROJECT_ROOT, "data", "artefacts", "cohort_selection"),
    "SINGLE_CELL": os.path.join(PROJECT_ROOT, "data", "artefacts", "single_cell_analysis"),
    "PSEUDOBULK":  os.path.join(PROJECT_ROOT, "data", "artefacts", "pseudobulk_preparation"),
    "DE":          os.path.join(PROJECT_ROOT, "data", "artefacts", "differential_expression"),
    "ENRICH":      os.path.join(PROJECT_ROOT, "data", "artefacts", "functional_enrichment"),

    # Autres dossiers du projet
    "RESULTS_R": os.path.join(PROJECT_ROOT, "Results_R_Analysis"),
    "TMP":       os.path.join(PROJECT_ROOT, "tmp_cache"),
    "DOCS":      os.path.join(PROJECT_ROOT, "documentation"),
}
for path in DIRS.values():
    os.makedirs(path, exist_ok=True)
os.chdir(PROJECT_ROOT)

# ==========================================================================================================
warnings.filterwarnings("ignore") 
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 6) 

# ==========================================================================================================
print(f"✅ Environnement chargé. Working directory: {os.getcwd()}")


✅ Environnement chargé. Working directory: c:\Z\M2_AIDA\TCGA_UCEC_project


## 2. Chargement des données

In [2]:
expr_filename = "TCGA-UCEC.star_counts.tsv.gz"
clin_filename = "TCGA-UCEC.clinical.tsv.gz"

expr_path = os.path.join(DIRS["RAW"], expr_filename)
clin_path = os.path.join(DIRS["RAW"], clin_filename)

expr = pd.read_csv(expr_path, sep="\t", index_col=0)
clin = pd.read_csv(clin_path, sep="\t")

# ==========================================================================================================
print(
    f"✅ Datasets successfully loaded\n"
    f"   - Expression matrix: {expr_path} | shape (genes × samples): {expr.shape}\n"
    f"   - Clinical matrix:   {clin_path} | shape (samples × variables): {clin.shape}"
)


✅ Datasets successfully loaded
   - Expression matrix: C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw\TCGA-UCEC.star_counts.tsv.gz | shape (genes × samples): (60660, 585)
   - Clinical matrix:   C:\Z\M2_AIDA\TCGA_UCEC_project\data\raw\TCGA-UCEC.clinical.tsv.gz | shape (samples × variables): (600, 79)


## 3. Harmonisation des données RNA-seq et cliniques au niveau patient

Cette étape vise à aligner les matrices d'expression RNA-seq et les données cliniques
sur un identifiant patient commun (barcode TCGA à 12 caractères).

- Les données RNA-seq sont indexées par **échantillons** (plusieurs par patient).
- Les données cliniques sont structurées au **niveau patient**.
- Une intersection directe sans harmonisation est donc invalide.

Le script :
1. identifie la colonne patient dans les données cliniques,
2. extrait les identifiants patient des deux sources,
3. calcule leur intersection,
4. filtre les deux tables pour ne conserver que les patients communs.

Cette harmonisation est une étape indispensable avant toute analyse conjointe.


In [3]:
clin_patient_col = "submitter_id"

expr_patient_ids = {c[:12] for c in expr.columns if c.startswith("TCGA-")}
clin_patient_ids = set(clin[clin_patient_col].astype(str).str.slice(0, 12))

common_patients = sorted(expr_patient_ids.intersection(clin_patient_ids))

clin_filtered = clin.set_index(clin_patient_col).loc[common_patients]
expr_filtered = expr.loc[:, [c for c in expr.columns if c[:12] in common_patients]]

print(
    f"✅ Harmonisation patient effectuée\n"
    f"   - Expression : {expr_filtered.shape[1]} échantillons "
    f"({expr_filtered.shape[0]} gènes)\n"
    f"   - Clinique   : {clin_filtered.shape[0]} patients "
    f"({clin_filtered.shape[1]} variables)\n"
    f"   - Patients communs : {len(common_patients)}"
)


✅ Harmonisation patient effectuée
   - Expression : 585 échantillons (60660 gènes)
   - Clinique   : 597 patients (78 variables)
   - Patients communs : 557


## 4. QC gènes — filtrage des gènes faiblement exprimés

Avant toute normalisation et modélisation, un contrôle qualité est appliqué
au niveau des gènes afin d’éliminer les gènes faiblement exprimés.

Un gène est conservé s’il présente une expression strictement supérieure à 1
dans au moins 20 % des échantillons.

**Justification du seuil :**
- un seuil minimal d’expression (> 1) permet d’exclure le bruit de comptage
  et les gènes quasi non exprimés ;
- l’exigence d’une expression dans au moins 20 % des échantillons permet
  de conserver des gènes présentant un signal suffisamment robuste à l’échelle
  de la cohorte, tout en évitant d’écarter des gènes potentiellement spécifiques
  de sous-groupes biologiques.

Ce compromis est couramment utilisé dans les analyses transcriptomiques
exploratoires et constitue un bon équilibre entre réduction du bruit et
préservation du signal biologique.

Le nombre de gènes avant et après QC est reporté afin de quantifier l’impact
de cette étape.


In [4]:
MIN_COUNT = 1
MIN_PROP = 0.2
min_samples = int(MIN_PROP * expr_filtered.shape[1])

expr_qc = expr_filtered.loc[(expr_filtered > MIN_COUNT).sum(axis=1) >= min_samples]

# ==========================================================================================================
print(
    f"✅ QC genes applique\n"
    f"   - Genes avant QC : {expr_filtered.shape[0]}\n"
    f"   - Genes apres QC : {expr_qc.shape[0]}\n"
    f"   - Genes filtres  : {expr_filtered.shape[0] - expr_qc.shape[0]}"
)


✅ QC genes applique
   - Genes avant QC : 60660
   - Genes apres QC : 31876
   - Genes filtres  : 28784


## 5. Normalisation des données d'expression

Après le QC gènes, une normalisation est appliquée afin de rendre les niveaux
d'expression comparables entre gènes avant l'apprentissage des modèles
de deep learning.

Les données fournies par Xena (STAR counts) étant déjà transformées en
log2(count + 1), aucune renormalisation de type CPM ou TPM n’est nécessaire.

Un **z-score par gène** est appliqué :
- centrage (moyenne nulle),
- réduction (variance unitaire).

Ce choix est adapté aux réseaux de neurones, qui sont sensibles à l’échelle
des variables, et permet d’éviter qu’un petit nombre de gènes à forte variance
ne domine l’apprentissage.


In [5]:
expr_norm = expr_qc.sub(expr_qc.mean(axis=1), axis=0)
expr_norm = expr_norm.div(expr_norm.std(axis=1) + 1e-8, axis=0)

# ==========================================================================================================
print(
    f"✅ Normalisation appliquee (z-score par gene)\n"
    f"   - Dimensions apres normalisation : {expr_norm.shape} (genes x echantillons)"
)


✅ Normalisation appliquee (z-score par gene)
   - Dimensions apres normalisation : (31876, 585) (genes x echantillons)


## 6. Sauvegarde des données pré-traitées

À l’issue du QC gènes et de la normalisation, les matrices finales sont
sauvegardées afin d’être réutilisées directement dans les notebooks de
modélisation.

Cette étape permet de :
- figer un jeu de données propre et reproductible,
- éviter de recalculer les étapes de pré-traitement,
- garantir une séparation claire entre pré-traitement et modélisation.

Les fichiers sauvegardés constituent l’entrée standard pour les modèles
supervisés et non supervisés du projet.


In [6]:
os.makedirs(DIRS["PROCESSED"], exist_ok=True)

expr_norm.to_csv(
    os.path.join(DIRS["PROCESSED"], "expr_norm_tcga_ucec.tsv"),
    sep="\t"
)
clin_filtered.to_csv(
    os.path.join(DIRS["PROCESSED"], "clin_tcga_ucec.tsv"),
    sep="\t"
)

# ==========================================================================================================
print(
    f"✅ Donnees sauvegardees\n"
    f"   - Expression normalisee : {os.path.join(DIRS['PROCESSED'], 'expr_norm_tcga_ucec.tsv')}\n"
    f"   - Donnees cliniques     : {os.path.join(DIRS['PROCESSED'], 'clin_tcga_ucec.tsv')}"
)


✅ Donnees sauvegardees
   - Expression normalisee : C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\expr_norm_tcga_ucec.tsv
   - Donnees cliniques     : C:\Z\M2_AIDA\TCGA_UCEC_project\data\processed\clin_tcga_ucec.tsv


In [7]:
print(
    f"Expression filtree : {expr_norm.shape} (genes x echantillons)\n"
    f"Clinique filtree   : {clin_filtered.shape} (patients x variables)"
)



Expression filtree : (31876, 585) (genes x echantillons)
Clinique filtree   : (597, 78) (patients x variables)


TCGA classique :

Certains patients ont de la clinique mais pas de RNA-seq utilisable
<Br>Le filtrage RNA-seq (low counts, QC) réduit l’effectif
<Br>Le filtrage clinique est indépendant

## 7. Prochaines étapes
- Décision analytique : **1 patient = 1 échantillon** (sélection d’un échantillon par patient ou agrégation)
- Identification et sélection des **labels TCGA** (POLE / MSI / CN-LOW / CN-HIGH) à partir de `clin_filtered`
- Split reproductible **train / validation / test** (seed fixe) + runs répétés / CV
- Baseline supervisée : **MLP minimal** (1 couche cachée) + métriques (ROC-AUC, F1, matrice de confusion)
- Modèle non supervisé : **autoencodeur** (dimension latente justifiée) + analyse de séparation des sous-types dans le latent
